## Addestramento di un **Percettrone Multistrato** (Perceptrone Multistrato, MLP, acronimo di Multilayer Perceptron) in **PyTorch**

* Applicheremo l'MLP per un compito di classificazione: Riconoscimento delle Cifre (Dati Immagine)
* Applicheremo l'MLP per un compito di regressione: ??? (Dati Tabulari)

[ENG]
* We will apply MLP for a classification task: Digit Recognition (Image Data)
* We will apply MLP for a regression task: ??? (Tabular Data)

### Dataset and Dataloaders

Prima di tutto, dobbiamo recuperare i dati di MNIST. Dobbiamo creare un **Dataset** e un **DataLoader** come abbiamo fatto nella lezione precedente. Dobbiamo creare i set di addestramento, validazione e test e un DataLoader per ognuno di essi.

[ENG]
First we need to retrieve the MNIST dataset.
We need to create Dataset and DataLoader as we did in previous lesson.
We need to create train, validation and test sets and a loader for each of them.


In [ ]:
import torch
import torchvision.transforms as T
def get_data(batch_size, test_batch_size=256):

  # Preparare le trasformazioni dei dati e poi combinarle sequenzialmente
  transform = list()
  # Converte un'immagine rappresentata come un array NumPy in un tensore PyTorch
  transform.append(T.ToTensor())
  # Sottrae la media specificata (mean=[0.5]) e quindi divide per la deviazione standard specificata (std=[0.5])
  # Porta tutti i valori del tensore nell'intervallo compreso tra -1 e 1.
  transform.append(T.Normalize(mean=[0.5], std=[0.5]))      # [ENG] Normalizes the Tensors between [-1, 1]
  # combina le trasformazioni definite in transform in una singola trasformazione composta.
  # In pratica, questo significa che quando applichi questa trasformazione composta a un'immagine, le due trasformazioni definite sopra verranno applicate in sequenza.
  transform = T.Compose(transform)

  # Carica il dataset di addestramento
  full_training_data = torchvision.datasets.MNIST('./data', train=True, transform=transform, download=True)
  # './data':  è la directory in cui il dataset verrà scaricato. Se non esiste, verrà creata.
  # train=True: specifica che questo dataset è per l'addestramento, quindi caricherà le immagini e le etichette di addestramento.
  # transform=transform: rappresenta l'insieme di trasformazioni delle immagini da applicare a ciascuna immagine durante il caricamento
  # download=True: indica al codice di scaricare il dataset da Internet se non è già disponibile in locale

  # Carica il dataset di test
  test_data = torchvision.datasets.MNIST('./data', train=False, transform=transform, download=True)
  # train=False: Nel caso del dataset MNIST, questa opzione indica il caricamento del dataset di test.

  # Crea divisioni per il training e la validazione
  # il set di addestramento che riceve circa il 70% dei dati e il set di convalida che riceve il resto
  num_samples = len(full_training_data)
  training_samples = int(num_samples*0.7+1) #Aggiungendo 1 si assicura che almeno un campione venga allocato al set di addestramento.
  validation_samples = num_samples - training_samples

  training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples, validation_samples])
  #torch.utils.data.random_split: è una funzione che suddivide casualmente un dataset in base alle dimensioni specificate per i set di addestramento e di convalida

  # Inizializza i dataloader
  train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True, num_workers=4)
  # shuffle=True:  il DataLoader mescola i dati di addestramento prima di suddividerli in batch
  # num_workers=4: utilizza 4 processi paralleli
  val_loader = torch.utils.data.DataLoader(validation_data, test_batch_size, shuffle=False, num_workers=4)
  test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False, num_workers=4)

  return train_loader, val_loader, test_loader


### Definizione della rete

<img src="https://miro.medium.com/v2/resize:fit:640/1*63sGPbvLLpvlD16hG1bvmA.gif" width="450"></br></br>

Un MLP è composto da due componenti:

**Fully-Connected Layers:** (Strati Completamente Connessi): Sono definiti come `torch.nn.Linear`.

**Activation function:** (Funzione di attivazione): Tra i livelli dobbiamo inserire una funzione di attivazione non lineare. Sono possibili molte scelte, vedi [doc](https://pytorch.org/docs/stable/nn.html).
Pero ora, useremo una funzione sigmoide (`torch.nn.Sigmoid`).

❗Non dimenticare che una rete deve estendere una classe `torch.nn.Module`.

Questo significa che quando si definisce una rete neurale in PyTorch, è necessario creare una classe che erediti dalla classe `torch.nn.Module`. Questo perché `torch.nn.Module` fornisce molte funzionalità utili per la definizione delle reti neurali, come la gestione automatica dei parametri, il calcolo automatico dei gradienti e la possibilità di annidare più moduli all'interno di una rete. Quindi, quando si definisce una rete, è importante farla ereditare da `torch.nn.Module` per sfruttare queste funzionalità.


[ENG]

### Network Definition

A MultiLayer Perceptron is made of two components:

**Fully-Connected Layers:** The fully-connected layers are defined as `torch.nn.Linear`. In practice a fully conected layer is an affine transformation:

$$
y = x \cdot W^T + b
$$

with two learnable set of parameters: the *weight*  $W$ and the *bias* $b$. \\
When defining a linear layer, we have to specify the number of input features and the number of output features, which are the dimensions of $x$ and $y$ respecively.

<img src="https://drive.google.com/uc?export=view&id=1hpqFqktOou8C4ZVOrJDiyYYG0b9PpPO2" width="450"></br></br>


**Activation function:** Between the layers we must put a non-linear activation. Multiple choices are possible, see [doc](https://pytorch.org/docs/stable/nn.html). For now let us use a sigmoid (`torch.nn.Sigmoid`).  

<img src="https://miro.medium.com/v2/resize:fit:970/1*Xu7B5y9gp0iL5ooBj7LtWw.png" width="450"></br></br>


❗❗❗ Do not forget that a network must extend a `torch.nn.Module`.

This means that when defining a neural network in PyTorch, you need to create a class that inherits from the  `torch.nn.Module ` class. This is because  `torch.nn.Module ` provides many useful features for defining neural networks, such as automatic parameter management, automatic calculation of gradients, and the ability to nest multiple modules within a network. So, when defining a network, it is important to have it inherit from  `torch.nn.Module ` to take advantage of these features.

In [ ]:
# Definizione della rete
class MLP(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim): #Prende in input le dimensioni dei layer di input, hidden e output
    super(MLP, self).__init__() # Questo chiama il costruttore della classe genitore (torch.nn.Module) per inizializzare la rete

    self.input_to_hidden = torch.nn.Linear(input_dim, hidden_dim) # Questa riga crea un modulo lineare che rappresenta il layer da input a hidden
    self.hidden_to_output = torch.nn.Linear(hidden_dim, output_dim) # Questa riga crea un altro modulo lineare che rappresenta il layer da hidden a output
    self.activation = torch.nn.Sigmoid() # Qui viene creato un modulo per l'attivazione dei neuroni, che in questo caso è la funzione sigmoide

  def forward(self, x): # Il metodo forward descrive come i dati passano attraverso la rete durante la fase di inoltro (forward pass)
    # Questo metodo è fondamentale per la fase di addestramento e di previsione del modello
    x = x.view(x.shape[0],-1) # Questa riga cambia la forma dei dati in ingresso x in modo che siano nella forma (batch_size, input_dim)
    # x.shape[0] è batch_size
    # -1 significa qualsiasi dimensione necessaria per mantenere invariato il numero totale di elementi originali.
    # (ENG) -1 refers to whatever size is necessary to maintain the original total number of elements.

    # Forward input through the layers
    x = self.input_to_hidden(x) # Questa riga applica la trasformazione lineare dal layer di input al layer hidden.
    x = self.activation(x) # l'output del layer hidden viene passato attraverso la funzione di attivazione sigmoide.
    x = self.hidden_to_output(x) # l'output del layer hidden (dopo l'attivazione sigmoide) viene ulteriormente trasformato per produrre l'output finale del modello.
    # (ENG) the output of the hidden layer (after sigmoid activation) is further transformed to produce the final model output.
    return x

## Loss/cost function

Per addestrare la rete, abbiamo bisogno di un cost/loss function. Il compito è la classificazione con multiple classi, quindi cost/loss function appropriata potrebbe essere cross-entropy con softmax.

Possiamo usare `torch.nn`, che contiene diverse cost/loss function, tra cui `torch.nn.CrossEntropyLoss`.

Nota che `torch.nn.CrossEntropyLoss` contiene già l'attivazione softmax, quindi non è necessario applicare il softmax all'output della nostra rete.

[ENG]

For training the network, we need a loss function. The task is classification with multiple classes, thus a proper loss could be a cross-entropy with softmax.

We can again use `torch.nn` which contains several losses, among which `torch.nn.CrossEntropyLoss`.

Notice that `torch.nn.CrossEntropyLoss` already contains the softmax activation, thus we do not need to apply the softmax to the output of our network.

In [ ]:
def get_cost_function():
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function

## Ottimizzatore (Optimizer)

Ora dobbiamo trovare un modo per aggiornare i parametri della nostra rete.
Questo può essere fatto con  [`torch.optim`](https://pytorch.org/docs/stable/optim.html), che contiene una vasta varietà di ottimizzatori.

[ENG]

Now, we need to find a way to update the parameters of our network.
This can be done with [`torch.optim`](https://pytorch.org/docs/stable/optim.html) which contains a large variety of optimizers.

In [ ]:
def get_optimizer(net, lr, wd, momentum): # net (un modello di rete neurale), lr (learning rate), wd (weigth decay), e momentum.
  optimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd, momentum=momentum) # utilizzando l'algoritmo di discesa del gradiente stocastico (SGD)
                                                                                           # (ENG) Stochastic gradient descent
  return optimizer # restituisce l'oggetto ottimizzatore

## Funzioni di Addestramento e Test

Le funzioni di addestramento e test devono:

1. Ciclare attraverso i dati (sfruttando il dataloader)
2. Eseguire il passaggio in avanti (forward pass) dei dati attraverso la rete
3. Calcolare *loss* per l'addestramento, *l'accuratezza* per il test o entrambi.

Inoltre, la funzione di addestramento deve:

1. Calcolare il gradiente con il passaggio all'indietro (`loss.backward()`)
2. Aggiornare *weights* (i pesi) utilizzando l'ottimizzatore (`optimizer.step()`)
3. Pulire il gradiente dei pesi in modo da non accumularlo (`optimizer.zero_grad()`)

Definiamo:

* **Iterazioni:** il numero di aggiornamenti del gradiente (cioè il numero di chiamate a `optimizer.step()`).
* **Epoca:** il numero di iterazioni che effettuiamo sull'intero dataset.


## Train and test functions

[ENG]

Training and test functions must:

1.  Loop over the data (exploiting the dataloader)
2.  Forward pass the data through the network
3.  Compute the loss for train, the accuracy for test or both.

Additionally, training function must:

1.   Compute the gradient with the backward pass (`loss.backward()`)
2.   Update the weights by using the optimizer (`optimizer.step()`)
3.   Clean the gradient of the weights not to accumulating it (`optimizer.zero_grad()`)

We define:
*   **Iterations:** the number of gradient updates (i.e. the number of calling the `optimizer.step()`).
*   **Epoch:** the number of iteratations we do over the whole dataset.

## Ricordiamo cosa è: Batch, Iterazione ed Epoca

* Batch: è un sottoinsieme dell'intero dataset utilizzato durante *una singola iterazione* di addestramento.
* Iterazione: Un'iterazione è un singolo aggiornamento dei pesi del modello utilizzando *un batch* di dati.
* Epoca: Un'epoca rappresenta *un passaggio completo* attraverso l'intero dataset di addestramento.

Esempio: Se abbiamo 1.000 esempi di addestramento e utilizziamo una dimensione di batch di 100, ci vorranno 10 iterazioni per elaborare un'epoca. Dopo la decima iterazione, avremo completato un'epoca, e il modello avrà analizzato tutti i 1.000 esempi di addestramento.

[ENG]

## RECAP: Batch, Iteration and Epoch

* Batch: is a subset of the entire dataset that is used during *one iteration* of training.
* Iteration: An iteration is a single update of the model's weights using *one batch* of data.
* Epoch: An epoch is *a complete pass* through the entire training dataset.

EXP: If you have 1,000 training examples and use a batch size of 100, it would take 10 iterations to process one epoch. After the 10th iteration, you will have completed one epoch, and the model will have seen all 1,000 training examples.

In [ ]:
# Ciclo di Addestramento (Training Loop)
def train_one_epoch(net, data_loader, optimizer, cost_function, device='cuda'): # "cuda" per l'uso della GPU
  samples = 0. # per traccia del numero totale di campioni durante l'epoca
  cumulative_loss = 0. # per traccia la somma cumulativa delle perdite durante l'epoca
  cumulative_accuracy = 0. # per traccia la somma cumulativa delle accuratezze durante l'epoca


  net.train() # Alcuni layer possono comportarsi in modo diverso durante l'addestramento rispetto alla fase di test.
              # (ENG) Strictly needed if network contains layers which has different behaviours between train and test

  for batch_idx, (inputs, targets) in enumerate(data_loader): # iterare sui batch di dati forniti da data_loader
    # Per ogni batch, i dati di input e i relativi target vengono caricati sulla GPU (Load data into GPU)
    inputs = inputs.to(device)
    targets = targets.to(device)

    # Passaggio in Avanti (Forward pass)
    outputs = net(inputs)

    # Applicazione della Funzione di Costo (Apply the loss)
    loss = cost_function(outputs,targets)

    # Passaggio all'Indietro (Backward pass)
    loss.backward()

    # Aggiornamento dei Parametri (Update parameters)
    optimizer.step()

    # Pulizia dei Gradiente (Clean/reset the gradients)
    optimizer.zero_grad()

    samples += inputs.shape[0] # traccia del numero totale di campioni che sono stati elaborati finora in questa epoca
    cumulative_loss += loss.item() # traccia della perdita totale accumulata durante l'epoca

    # outputs.max(dim=1) restituisce due tensori: il primo contiene i valori massimi lungo la dimensione 1
    # che corrispondono alle probabilità predette per ciascuna classe
    # il secondo contiene gli indici dei massimi valori che rappresentano le classi predette.
    # Stiamo scartando il primo tensore (usando _) e conservando gli indici delle classi predette.
    _, predicted = outputs.max(dim=1) # (ENG) returns (maximum_value, index_of_maximum_value)

    # stiamo confrontando gli indici delle classi predette (predicted) con gli indici delle classi target reali (targets) per il batch corrente.
    # La funzione predicted.eq(targets) restituisce un tensore di valori booleani che indicano se le predizioni sono corrette (True) o sbagliate (False).
    # Usando .sum(), contiamo quanti di questi valori booleani sono True (cioè quante predizioni sono corrette) e quindi convertiamo il risultato in un valore scalare con .item().
    # Questo valore scalare rappresenta il numero di predizioni corrette nel batch corrente.
    cumulative_accuracy += predicted.eq(targets).sum().item()

  # la funzione restituisce due valori: la perdita media (cumulative_loss diviso per il numero totale di esempi)
  # e l'accuratezza media (cumulative_accuracy diviso per il numero totale di esempi, moltiplicato per 100 per ottenere la percentuale)
  return cumulative_loss/samples, cumulative_accuracy/samples*100

Durante l'addestramento, è necessario testare il modello sul set di validazione per essere sicuri di migliorare le prestazioni.

[ENG] While training, we need to test our model on the validaiton set to be sure we are improving the model performance.

In [ ]:
# Fase di Validazione (Validation Step)
def validation_step(net, data_loader, cost_function, device='cuda'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  net.eval()  # Alcuni layer possono comportarsi in modo diverso durante l'addestramento rispetto alla fase di test.
              # (ENG) Strictly needed if network contains layers which has different behaviours between train and test

  # torch.no_grad(): significa qualsiasi operazione che viene eseguita all'interno di questo contesto non influirà sui gradienti del modello.
  # In the following steps the gradient of the model will not be affected.
  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader): # iteriamo attraverso i batch di dati di convalida
      # Per ciascun batch, carichiamo i dati nella GPU (Load data into GPU)
      inputs = inputs.to(device)
      targets = targets.to(device)

      # Passaggio in Avanti (Forward pass)
      outputs = net(inputs)

      # Applicazione della Funzione di Costo (Apply the loss)
      loss = cost_function(outputs, targets)

      samples+=inputs.shape[0]
      cumulative_loss += loss.item() # convertiamo il risultato (tensore) in un valore scalare con ".item()"
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


## Combiniamo il tutto (FUNZIONE PRINCIPALE)

Infine, abbiamo bisogno di una funzione principale che:

1) Inizializza tutto quello che abbiamo definito,

2) Definisce gli iperparametri (*hyperparameters*),

3) Esegue il ciclo su più epoche,

4) Stampa i risultati.


[ENG]
## Let's combine everything (MAIN FUNCTION)

Finally, we need a main function which

1) Initializes everything,

2) Defines hyperparameters,

3) Loops over multiple epochs,

4) Print the results.

In [ ]:
import torchvision
import torch

# Definire tutti gli iperparametri (batch_size..... epochs)
def main(batch_size=128, input_dim=28*28, hidden_dim=100, output_dim=10, device='cuda:0', learning_rate=0.01, weight_decay=0.000001, momentum=0.9, epochs=10):

  # Ottiene i DataLoaders
  train_loader, val_loader, test_loader = get_data(batch_size)

  # Inizializza la rete e la sposta sul dispositivo scelto (GPU).
  # (ENG) Initialize the network and moves it to GPU.
  net = MLP(input_dim, hidden_dim, output_dim).to(device)

  # Istanzia l'ottimizzatore
  # (ENG) Instantiates the optimizer
  optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)

  # Crea la funzione di costo
  # (ENG) Creates the cost function
  cost_function = get_cost_function()

  n_iterations = 0
  # Per ogni epoca, addestra la rete e poi calcola i risultati della valutazione.
  # (ENG) For each epoch, train the network and then compute evaluation results
  for e in range(epochs):
    train_loss, train_accuracy = train_one_epoch(net, train_loader, optimizer, cost_function)
    val_loss, val_accuracy = validation_step(net, val_loader, cost_function)

    print('Epoch: {:d}'.format(e+1)) # Questo è un modo comune per tenere traccia dell'epoca corrente
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')

In [ ]:
# Chiama la funzione principale
main()

Ci si aspetta che ad ogni epoca la perdita durante l'addestramento diminuisca, mentre l'accuratezza durante la validazione aumenti.

[ENG]
It is expected that at each epoch the loss during training decreases, while the accuracy during validation increases.

Tracciamo le curve di addestramento e di validazione per ciascuna epoca.
Per farlo, definiamo la funzione main2.

[ENG] Let's plot the training and validation curves for each epoch. To do so we define main2.

In [ ]:
import torchvision
import torch
from matplotlib import pyplot as plt

# Definire tutti gli iperparametri (batch_size..... epochs)
def main2(batch_size=128, input_dim=28*28, hidden_dim=100, output_dim=10, device='cuda:0', learning_rate=0.01, weight_decay=0.000001, momentum=0.9, epochs=10):

  trainingEpoch_loss = []
  validationEpoch_loss = []
  validationEpoch_accuracy = []

  # Ottiene i DataLoaders
  train_loader, val_loader, test_loader = get_data(batch_size)

  # Inizializza la rete e la sposta sul dispositivo scelto (GPU).
  # (ENG) Initialize the network and moves it to GPU.
  net = MLP(input_dim, hidden_dim, output_dim).to(device)

  # Istanzia l'ottimizzatore
  # (ENG) Instantiates the optimizer
  optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)

  # Crea la funzione di costo
  # (ENG) Creates the cost function
  cost_function = get_cost_function()

  n_iterations = 0
  # Per ogni epoca, addestra la rete e poi calcola i risultati della valutazione.
  # (ENG) For each epoch, train the network and then compute evaluation results
  for e in range(epochs):
    train_loss, train_accuracy = train_one_epoch(net, train_loader, optimizer, cost_function)
    val_loss, val_accuracy = validation_step(net, val_loader, cost_function)

    trainingEpoch_loss.append(train_loss) # Raccogla la perdita di allenamento
    validationEpoch_loss.append(val_loss) # Raccogla la perdita della convalida
    # validationEpoch_accuracy.append(val_accuracy) # Raccogla l'accuratezza della convalida


  plt.plot(trainingEpoch_loss, label='training loss')
  plt.plot(validationEpoch_loss,label='validation loss')
  # plt.plot(validationEpoch_accuracy,label='validation accuracy')
  plt.title('Training and Validation Curves')
  plt.xlabel('Epoch')
  plt.legend()
  plt.show


In [ ]:
main2()

# Esercizio 1

**1)** Esegui lo stesso codice quando:

* batch_size=128
* input_dim=28*28
* hidden_dim=100
* output_dim=10
* device='cuda:0'
* learning_rate=0.01
* weight_decay=0.000001
* momentum=0.9
* epoche=10
* con l'ottimizzatore Adam: `optimizer = torch.optim.Adam(net.parameters())`.

**2)** Utilizza `torch.save(net.state_dict(), "model.ckpt")`.

**ATTENZIONE:** `torch.save(net.state_dict(), "model.ckpt")` è una riga di codice che salva i pesi (o i parametri) di una rete neurale PyTorch nel file `model.ckpt`. Questo permette di conservare i parametri addestrati della rete in modo da poterli riutilizzare in futuro, ad esempio per fare previsioni o per continuare l'addestramento. Quando si desidera utilizzare la rete in un'altra sessione o in un altro momento, è possibile caricare questi pesi utilizzando la funzione `load_state_dict()` e inizializzare una rete con i pesi precedentemente addestrati. Questo è particolarmente utile quando si lavora su progetti di deep learning che richiedono molto tempo per l'addestramento e si desidera salvare i progressi o condividere modelli con altri.

**3)** Carica i pesi salvati e addestra ed esegui la valutazione per 5 epoche con gli stessi iperparametri.

Per eseguire ciò:

a) Il modello deve essere creato.

b) Il modello deve essere spostato sul dispositivo specificato ('cuda:0') utilizzando `net = net.to(device)`.

c) I pesi sono caricati nel modello utilizzando `net.load_state_dict`.

d) L'ottimizzatore è creato con il tasso di apprendimento (learning rate) e weigth decay.

e) Sia il passaggio di addestramento che quello di validazione passano il dispositivo come argomento.

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

[ENG]

**1)** Run the same code when:

* batch_size=128
* input_dim=28*28
* hidden_dim=100
* output_dim=10
* device='cuda:0'
* learning_rate=0.01
* weight_decay=0.000001
* momentum=0.9
* epochs=10
* with Adam optimizer optimizer = `torch.optim.Adam(net.parameters())`.

**2)** Use `torch.save(net.state_dict(), "model.ckpt")`.


**IMPORTANT:** `torch.save(net.state_dict(), "model.ckpt")` is a line of code that saves the weights (or parameters) of a PyTorch neural network to the `model.ckpt` file. This allows you to keep the trained parameters of the network so you can reuse them in the future, for example to make predictions or to continue training. When you want to use the network in another session or at another time, you can load these weights using the `load_state_dict()` function and initialize a network with the previously trained weights. This is especially useful when you're working on deep learning projects that require a lot of training time and you want to save your progress or share models with others.

**3)** Load the saved weigths and train and evaluate 5 epochs with the same hyperparameters.

To perform this:

a) The model should be created

b) The model should be moved to the specified device ('cuda:0') using `net = net.to(device)`.

c) The weights are loaded into the model using `net.load_state_dict`.

d) The optimizer is created with the specified learning rate and weight decay.

e) Both the training and validation steps pass the device as an argument.

# Esercizio 2

## Eseguire una task di classificazione con DATI TABULARI (dataset del Titanic) usando MLP.

* Scarica il dataset del Titanic dall'URL fornito: ``` https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv ```
* Si tratta di un compito di classificazione con due classi in cui le etichette sono memorizzate in 'Survived'.
* Come metodo di classificazione, utilizzerai MLP.
* Utilizzerai tutte le caratteristiche fornite, che includono: Pclass (categorical), Name, Sex (Categorical), Age,  Siblings/Spouses Aboard, Parents/Children Aboard and Fare.
* Rimuovi le colonne non necessarie: eliminiamo solo "Name" anche se molte altre colonne potrebbero essere irrilevanti.
* Applica la codifica one-hot per le variabili categoriali.
* Elimina gli esempi con valori mancanti (NA).``` dropna() ```
* Dividi i dati in modo che il set di addestramento contenga l'80% e il set di test il 20% del numero totale di dati. Non è necessario applicare la cross-validation. Dato che si tratta di un dataset molto piccolo, non è necessario utilizzare batch.
* Standardizza le caratteristiche (la standardizzazione è un tipo di normalizzazione).
```
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
```
* Definisci un modello MLP personalizzato in modo che abbia uno strato (livello) di input, due strati (livelli) nascosto e uno strato di output. Usa la funzione di attivazione ReLU in modo appropriato. ```nn.ReLU()```

* Scrivi la funzione principale (main) quando:
```
input_dim = X_train.shape[1]
hidden_dim1 = 64
hidden_dim2 = 32
output_dim = 2  # 2 classes: Survived or not
```
* Definisci una funzione di costo che ritieni adatta per questo compito.
* Scegli un ottimizzatore con un "learning rate" che ritieni appropriato.
* Nota che i dati sono TABULAR, quindi è necessario convertirli in tensori PyTorch per il processo.
```
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
```
* Addestra il classificatore per 100 epoche.
* Testa il modello addestrato con i dati di test e stampa l'accuratezza finale.





[ENG]

## Performing a classification task with TABULAR DATA (titanic dataset) using MLP.

* Download the titanic dataset from the supplied URL: ``` https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv ```
* This is a classification task with two classes in which the labels are stored in “Survived”.
* As the classification method you will be using MLP.
* You will be using all features supplied which are: Pclass (categorical), Name, Sex (Categorical), Age,  Siblings/Spouses Aboard, Parents/Children Aboard and Fare.
* Drop unnecessary columns: we drop only “Name” even though many other column can be irrelevant.
* Apply one-hot encoding for the categorical variables.
* Drop examples with NA values. ``` dropna() ```
* Split the data such that train has 80% and test is 20% of the total number of data. You do not need to apply cross validation. Since this is a very small dataset, we do not need to use batch.
* Standardize features (standardization is a kind of normalization).
```
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
```
- Define a custom MLP model such that it has an input layer, a hidden layer and an output layer. Use ReLU in an appropriate way. ```nn.ReLU()```
- Write the main function when
```
input_dim = X_train.shape[1]
hidden_dim1 = 64
hidden_dim2 = 32
output_dim = 2  # 2 classes: Survived or not
```
-	Define a cost function that you believe it is suitable for this task.
- Pick an optimizer with a learning rate you believe appropriate.
- Note that the data is TABULAR so you need to convert it to PyTorch tensors to process.
```
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
```
-	Train the classifier for 100 epochs.
-	Test the trained model with testing data and print out the final accuracy.